# Unconstrained Optimization

## Direct Search Method

#### Univariate Method

In [7]:
from sage.all import var, solve
from sage.all import *

In [8]:
var('x y z')
f = function('f', nargs=2)
f(x, y) == 6*x**2 - 6*x*y + 2*y**2 - x - 2*y
def usme1(f, x0, y0, e = 0.01):
    z0 = (x0, y0)
    f0 = f(*z0)
    e1 = (1, 0)
    fxp = f(z0[0] + e * e1[0], z0[1])
    fxn = f(z0[0] - e * e1[0], z0[1])
    t = var('t')
    if fxp < fxn:
        if fxp < f0:
            g = f(z0[0] + t * e1[0], z0[1])
            T = solve(g.diff(t) == 0, t)
            if T:
                l = T[0].evalf()
                z1 = (z0[0] + l * e1[0], z0[1])
                return z1
    elif fxn < fxp:
        if fxn < f0:
            g = f(z0[0] - t * e1[0], z0[1])
            T = solve(g.diff(t) == 0, t)
            if T:
                l = T[0].evalf()
                z1 = (z0[0] - l * e1[0], z0[1])
                return z1
    return z0

def usme2(f, x0, y0, e = 0.01):
    z0 = (x0, y0)
    f0 = f(*z0)
    e2 = (0, 1)
    fyp = f(z0[0], z0[1] + e * e2[1])
    fyn = f(z0[0], z0[1] - e * e2[1])
    t = var('t')
    if fyp < fyn:
        if fyp < f0:
            g = f(z0[0], z0[1] + t * e2[1])
            T = solve(g.diff(t) == 0, t)
            l = T[0].evalf()
            z1 = (z0[0], z0[1] + l * e2[1])
            return z1
    elif fyn < fyp:
        if fyn < f0:
            g = f(z0[0], z0[1] - t * e2[1])
            T = solve(g.diff(t) == 0, t)
            l = T[0].evalf()
            z1 = (z0[0], z0[1] - l * e2[1])
            return z1
    return z0

In [9]:
def univariatesearch(f, z0, usme1, usme2, maxitr=100, tol=1e-10):
    i = 0
    error = float('inf')
    while i < maxitr and error > tol:
        z1 = usme1(f, z0[0], z0[1])
        z2 = usme2(f, z1[0], z1[1])
        if z1 is None or z2 is None:
            print("Error: one of the intermediate points is None.")
            break
        error = ((z1[0] - z2[0])**2 + (z1[1] - z2[1])**2)**0.5
        z0 = z2
        i += 1
    return z2
z0 = (0, 0)
univariatesearch(f, z0, usme1, usme2)

(0, 0)

#### Powell's Method

In [150]:
z0 = (0, 0)
t = symbols('t')
def special(f, z0, usme1, usme2):
        z1 = usme1(f, z0[0], z0[1])
        z2 = usme2(f, z1[0], z1[1])
        error = ((z2[0] - z0[0])^2 + (z2[1] - z0[1])^2)^0.5
        return (z2[0] - z0[0], z2[1] - z0[1]), error
        
def powell(f, z0, usme1, usme2, special, maxitr = 100, tol = 1e-20):
    i = 0
    error = float('inf')
    while i < maxitr and error > tol:
        sp, error = special(f, z0, usme1, usme2)
        print(f"special direction{sp}, error is {error}")#no problem # remove print
        g = f(z0[0] + t * sp[0], z0[1] + t * sp[1])
        T = solve(g.diff(t) == 0, t)
        print(f"optimal step length{T}") #no problem # remove print
        if T:
            l = T[0].evalf()
            print(f"l{l}")
            z3 = (z0[0] + l * sp[0], z0[1] + l * sp[1])
            z0 = z3
            print(f"the z3 is{z0}") # some problem here ?! 
        else:
            print("No solution found for optimal step length.")
            break
        i += 1
    return z0
powell = powell(f, z0, usme1, usme2, special)
print(f"the result is{powell}")

special direction(0.0833333333333333, 0.625000000000000), error is 0.630531081267565
optimal step length[1.30612244897959]
l1.30612244897959
the z3 is(0.108843537414966, 0.816326530612245)
special direction(0.382653061224490, 0.420918367346939), error is 0.568854671453515
optimal step length[2.81318681318681]
l2.81318681318681
the z3 is(1.18531808327727, 2.00044853106077)
special direction(-0.101760484413545, 0.124887867234807), error is 0.161096789448263
optimal step length[0.775170325510978]
l0.775170325510978
the z3 is(1.10643637545026, 2.09725789975755)
special direction(0.0255259077618457, 0.100685525060612), error is 0.103870818441935
optimal step length[1.87976135842133]
l1.87976135842133
the z3 is(1.15441899049961, 2.28652265911885)
special direction(0.0721756723931513, 0.0533693352202866), error is 0.0897642112829987
optimal step length[1.83491992384363]
l1.83491992384363
the z3 is(1.28685556979061, 2.38445111563685)
special direction(-0.0112966786388562, 0.0288872210907885), 

In [7]:
import numpy as np

def assignsimplexvalues(simplex, f):
    f_values = np.array([f(*vertex) for vertex in simplex])
    indices = np.argsort(f_values)
    sorted_simplex = np.array(simplex)[indices]
    sorted_f_values = f_values[indices]
    B = sorted_simplex[0]
    G = sorted_simplex[1]
    W = sorted_simplex[2]
    fB = sorted_f_values[0]
    fG = sorted_f_values[1]
    fW = sorted_f_values[2]
    return B, G, W, fB, fG, fW

def neldermead(f, initial_simplex, tol=1e-10, max_iter=500):
    simplex = np.array(initial_simplex)
    f_values = np.array([f(*vertex) for vertex in simplex])
    error = tol + 1
    for i in range(max_iter):
        if error <= tol:
            break
        B, G, W, fB, fG, fW = assignsimplexvalues(simplex, f)
        M = (B + G) / 2
        R = 2 * M - W  # reflect pt
        fR = f(*R)
        if fR < fB:
            E = 2 * R - M  # ext
            fE = f(*E)
            if fE < fB:
                W = E
            else:
                W = R
        else:
            if fR < fG:
                W = R
            elif fR > fG:
                c1 = (M + R) / 2
                c2 = (M + W) / 2
                C = c1 if f(*c1) < f(*c2) else c2
                if f(*C) > fW:  # contract
                    S = (B + W) / 2
                    G = (B + G) / 2
                    W = S  # shrink
                else:
                    W = C
        simplex = [B, G, W]
        error = np.linalg.norm(W - B)
    return B, f(*B)
B = np.array([-10, 10])
G = np.array([-5, -7])
W = np.array([10.0, -10.0])
simplex = [B, G, W]
optimal_point, optimal_value = neldermead(f, simplex)
print("Optimal Point:", optimal_point)
print("Optimal Value:", optimal_value)

/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:18: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:4: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:54: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See 

Optimal Point: [1.33333331 2.49999995]
Optimal Value: -3.166666666666668


In [4]:
import random
from sage.all import var, cos, sin, sqrt, pi, solve

def rwalk(f, x0, y0, epsilon=1e-5, max_iter=1000):
    x, y, t = var('x y t')
    gradx = f.diff(x)
    grady = f.diff(y)
    def gradient(xval, yval):
        return [gradx.evalf(subs={x: xval, y: yval}), grady.evalf(subs={x: xval, y: yval})]
    def steplen(x_val, y_val, direction):
        t = var('t')
        g = f.subs({x: x_val + t * direction[0], y: y_val + t * direction[1]})
        T = solve(g.diff(t) == 0, t)
        return float(T[0]) if T else 1.0
    xn, yn = x0, y0
    for _ in range(max_iter):
        grad = gradient(xn, yn)
        if sqrt(grad[0]**2 + grad[1]**2) < epsilon:
            break
        rangle = random.uniform(0, 2 * pi)
        rdir = [cos(rangle), sin(rangle)]
        stepsize = steplen(xn, yn, d)
        d = [-grad[0] + stepsize * rdir[0], -grad[1] + stepsize * rdir[1]]
        norm = sqrt(d[0]**2 + d[1]**2)
        d = [di / norm for di in d]
        xno = xn + stepsize * d[0]
        yno = yn + stepsize * d[1]
        xn, yn = xno, yno
    return xn, yn

x, y = var('x y')
f = 6*x**2 - 6*x*y + 2*y**2 - x - 2*y

x0, y0 = 0.0, 0.0

result = rwalk(f, x0, y0)
print(f"Optimized point: {result}")


AttributeError: 'sage.symbolic.expression.Expression' object has no attribute 'evalf'